In [1]:
#安裝相關函式庫
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
from lxml import etree

In [3]:
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    }

In [4]:
#請求網頁
def requestHtml(url):
  s = requests.Session()
  r = s.get(url, headers=headers_Get)
  r.encoding = r.apparent_encoding
  #print(r.text)
  return etree.HTML(r.text)

In [6]:
#限定搜尋結果(指定網站)
url = 'https://www.google.com/search?q=%E7%BE%8E%E5%9C%8B%E9%8A%80%E8%A1%8C+site:cw.com.tw&tbm=nws'
dom = requestHtml(url)

articleLinks = dom.xpath('//div[@class="SoaBEf"]//a/@href')
links = []
for link in articleLinks:
  if link[22:29] == "article":
      links.append(link)

print(links)

['https://www.cw.com.tw/article/5125190', 'https://www.cw.com.tw/article/5125155', 'https://www.cw.com.tw/article/5125137', 'https://www.cw.com.tw/article/5125075', 'https://www.cw.com.tw/article/5125060', 'https://www.cw.com.tw/article/5125052', 'https://www.cw.com.tw/article/5125159', 'https://www.cw.com.tw/article/5125096']


In [7]:
#下一頁 
#//span[text()='下一頁']/parent::a/@href
#//span[contains(text(),'下一頁')]
nextPage = dom.xpath('//a[@id="pnnext"]/@href')
print(nextPage)

['/search?q=%E7%BE%8E%E5%9C%8B%E9%8A%80%E8%A1%8C+site:cw.com.tw&tbm=nws&ei=eU4hZPD8MNrikPIPs56B4Ac&start=10&sa=N&ved=2ahUKEwiwnci90_v9AhVaMUQIHTNPAHwQ8NMDegQIAxAW']


In [8]:
#這樣只能一頁
if nextPage:
  dom = requestHtml('https://www.google.com' + nextPage[0])

  articleLinks = dom.xpath('//div[@class="SoaBEf"]//a/@href')
  for link in articleLinks: 
    if link[22:29] == "article":
      links.append(link)

print(links)

['https://www.cw.com.tw/article/5125190', 'https://www.cw.com.tw/article/5125155', 'https://www.cw.com.tw/article/5125137', 'https://www.cw.com.tw/article/5125075', 'https://www.cw.com.tw/article/5125060', 'https://www.cw.com.tw/article/5125052', 'https://www.cw.com.tw/article/5125159', 'https://www.cw.com.tw/article/5125096', 'https://www.cw.com.tw/article/5125063', 'https://www.cw.com.tw/article/5125196', 'https://www.cw.com.tw/article/5125136', 'https://www.cw.com.tw/article/5122430', 'https://www.cw.com.tw/article/5125016', 'https://www.cw.com.tw/article/5125188', 'https://www.cw.com.tw/article/5125117', 'https://www.cw.com.tw/article/5124891']


In [9]:
import time
import random
wait_time = random.uniform(1, 3)
nextPage = dom.xpath('//a[@id="pnnext"]/@href')
#當有下一頁時，就一直爬取
while nextPage:
  dom = requestHtml('https://www.google.com' + nextPage[0])

  articleLinks = dom.xpath('//div[@class="SoaBEf"]//a/@href')
  for link in articleLinks:
    if link[22:29] == "article":
      links.append(link)

  print(len(links))
  time.sleep(wait_time)

  nextPage = dom.xpath('//a[@id="pnnext"]/@href')



24
32
40
48
55
64
71
79
88
97
105
112
121
129
136
145
153
162
170
179
186
194
201
209
218
226
235
244


In [10]:
for link in links:
  print(link)

https://www.cw.com.tw/article/5125190
https://www.cw.com.tw/article/5125155
https://www.cw.com.tw/article/5125137
https://www.cw.com.tw/article/5125075
https://www.cw.com.tw/article/5125060
https://www.cw.com.tw/article/5125052
https://www.cw.com.tw/article/5125159
https://www.cw.com.tw/article/5125096
https://www.cw.com.tw/article/5125063
https://www.cw.com.tw/article/5125196
https://www.cw.com.tw/article/5125136
https://www.cw.com.tw/article/5122430
https://www.cw.com.tw/article/5125016
https://www.cw.com.tw/article/5125188
https://www.cw.com.tw/article/5125117
https://www.cw.com.tw/article/5124891
https://www.cw.com.tw/article/5124860
https://www.cw.com.tw/article/5125173
https://www.cw.com.tw/article/5125085
https://www.cw.com.tw/article/5125078
https://www.cw.com.tw/article/5125175
https://www.cw.com.tw/article/5125182
https://www.cw.com.tw/article/5125053
https://www.cw.com.tw/article/5125002
https://www.cw.com.tw/article/5125035
https://www.cw.com.tw/article/5125011
https://www.

In [11]:
print("原始筆數：{0}".format(len(links)))
#去除重複
links = list(dict.fromkeys(links))
print("去除重複後筆數：{0}".format(len(links)))

原始筆數：244
去除重複後筆數：227


In [12]:
dom = requestHtml(links[5])
#取得標題、內文、發文日期、圖檔連結
title = dom.xpath('//h1/text()')
image = dom.xpath('//div[@class="article__img"]/span/@data-zoom')
publish_date = dom.xpath('//time/text()')
content = dom.xpath('//div[contains(@class,"article__detail")]/following-sibling::node()[2]/p/text()')
tags = dom.xpath('//a[@class="keywords"]/@data-keyword')
#連接 list 內的值，使用斷行 \n
content = '\n'.join(content)
#組成dict
article = {
    "title": title,
    "image": image,
    "publish_date": publish_date,
    "content": content
}

print(article['content'])

台積電創辦人張忠謀與《晶片戰爭》作者、美國塔夫茨大學副教授米勒（Chris Miller），3月16日一起來到《天下》主辦的「半導體世紀對談」。《天下》訪問幾位在現場、線上聆聽的前台積電主管與分析師，在這場笑聲與掌聲不斷的一個多小時談話中，歸納出三個亮點。
這場論壇的第一個亮點，就是他語出驚人。
他先主動問小他50多歲的美國學者，對於未來5年或10年，在兩岸、中美都不會發生戰爭的前提下，半導體供應鏈會變得如何？
米勒小心翼翼地給了個四平八穩的回答。張忠謀接下來，卻開始評論起美國對中國的半導體禁令，「我對這個沒有意見，事實上，應該說，我支持這個政策。」


In [13]:
def composeArticle(dom, link):
  #取得標題、內文、發文日期、圖檔連結
  title = dom.xpath('//h1/text()')
  image = dom.xpath('//div[@class="article__img"]/span/@data-zoom')
  publish_date = dom.xpath('//time/text()')
  content = dom.xpath('//div[contains(@class,"article__detail")]/following-sibling::node()[2]/p/text()')
  tags = dom.xpath('//a[@class="keywords"]/@data-keyword')
  #連接 list 內的值，使用斷行 \n
  content = '\n'.join(content)

  if len(title) == 0:
    return {}

  article_image = ''
  if image:
    article_image = image[0]

  #組成dict
  return {
    "title": title[0],
    "image": article_image,
    "publish_date": publish_date[0],
    "content": content,
    "link": link
  }

In [14]:
from random import randint
from time import sleep

In [15]:
articles = []
#試作時取前10筆，實際要全部擷取文章時，將[:10]移除
for link in links[:10]:
  dom = requestHtml(link)

  article = composeArticle(dom, link)
  print(article)
  articles.append(article)

  sleep(randint(1,5))

{'title': '分析師：不，德意志銀行不是下一個瑞士信貸', 'image': 'https://storage.googleapis.com/www-cw-com-tw/article/202303/article-641e62a0042b3.jpg', 'publish_date': '\n發布時間：2023-03-25\n', 'content': '3月8日，Silvergate Bank宣布將逐步結束營業、清算資產。3月10日，矽谷銀行（Silicon Valley Bank）母公司SVB揭露鉅額虧損及募資計畫，股價血崩，不但矽谷銀行陷入流動性危機，新創圈人心惶惶，也帶崩一眾美國金融股。美國監管當局承諾致力保護金融穩定，接管矽谷銀行，保證儲戶安全。3月12日，紐約監管當局直接勒令關閉Signature Bank。\n無巧不巧，歐洲也有銀行爆雷，近年醜聞纏身的瑞士信貸（Credit Suisse）出現鉅額虧損及財務危機，價值160億瑞郎的瑞信債券減記為零，重創瑞士銀行業形象與信譽。\n瑞士聯邦委員會逼瑞銀集團出手收購瑞信，並提供90億瑞郎擔保給瑞銀，確保收購無虞。\n美國不但擔保矽谷銀行和Signature Bank存戶的存款不會有事，也願意提供有需要的銀行緊急借款。\n但政府的背書，不足以讓所有投資人安心，依然有許多人擔心恐慌和災禍蔓延。\n不同於2008年金融危機，這次的問題並非在於銀行資產的品質。當前的信心危機主因是矽谷銀行在風控不足情況下，採取梭哈長債、押注利率的投資策略，結果大翻車，資產流出速度過快，被迫實現債券的浮虧並宣布增資，進而引發擠兌。\n投資人認為，以美國區域銀行為主，有很多銀行都做了類似SVB的投資策略，所以很擔心若客戶想要提款，這些銀行是否有足夠的資本支應虧損。市場的目光焦點目前集中在被3家主要信評機構降評的第一共和銀行上。\n穆迪就不看好第一共和銀行能不賣資產而撐住擠兌，在報告中指出，「儘管第一共和銀行每天存款流出的速度正在大幅減慢，穆迪認為2022年Q4存款有68%是未保險的，而這間銀行已經損失大筆存款，侵蝕了這間銀行的經營強度」。\n穆迪認為，第一共和銀行新獲得的資金多屬相對短期的性質，最終勢必得出售資產來償付。\n第一共和銀行主要迎合有錢客戶，絕大多數客戶的存款不受美國聯邦存款保險公司（Federal Deposit I

In [46]:
import csv

csv_file = '天下雜誌_美國銀行.csv'
csv_columns = ['title', 'image', 'publish_date', 'content', 'link']
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in articles:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [48]:
#將 dict 轉換成 json 字串
import json
# Serializing json  
json_object = json.dumps(articles, indent = 4, ensure_ascii=False).encode('utf-8').decode('utf-8')
json_file = '天下雜誌_美國銀行.json'
with open(json_file, 'w', encoding='utf-8') as f:
    f.write(json_object)